In [ ]:
from pathlib import Path

import numpy as np
import xarray as xr

import pymc as pm
import arviz as az

import hvplot.xarray

# laod cherab-generomak related information
from cherab.generomak.equilibrium import load_equilibrium as load_generomak_equilibrium

import holoviews as hv
hv.extension("bokeh")

Load measured spectra and generomak equilibrium for visualisations. Fill in the path to the datafile, e.g. "o2eadp-2025-Norris/data/cxrs/123456.nc".

In [2]:
datafile_path = Path("")
ds_cxrs = xr.open_dataset(datafile_path)

equilibrium = load_generomak_equilibrium()

Precalculate some additional data for plotting 

In [3]:
# add cartesian spot coordinates
ds_cxrs.coords["spot_x"] = (
    "channel",
    ds_cxrs.spot_r.values * np.cos(np.deg2rad(ds_cxrs.spot_phi.values)),
)
ds_cxrs.coords["spot_y"] = (
    "channel",
    ds_cxrs.spot_r.values * np.sin(np.deg2rad(ds_cxrs.spot_phi.values)),
)

#
n_toroidal_points = 360
phi = np.linspace(0, 2 * np.pi, n_toroidal_points)

generomak_data = xr.Dataset(
    {
        "lcfs_x_mid_out": ("phi", equilibrium.lcfs_polygon[:, 0].min() * np.cos(phi)),
        "lcfs_y_mid_out": ("phi", equilibrium.lcfs_polygon[:, 0].min() * np.sin(phi)),
        "lcfs_x_mid_in": ("phi", equilibrium.lcfs_polygon[:, 0].max() * np.cos(phi)),
        "lcfs_y_mid_in": ("phi", equilibrium.lcfs_polygon[:, 0].max() * np.sin(phi)),
        "fw_x_mid_out": ("phi", equilibrium.limiter_polygon[:, 0].min() * np.cos(phi)),
        "fw_y_mid_out": ("phi", equilibrium.limiter_polygon[:, 0].min() * np.sin(phi)),
        "fw_x_mid_in": ("phi", equilibrium.limiter_polygon[:, 0].max() * np.cos(phi)),
        "fw_y_mid_in": ("phi", equilibrium.limiter_polygon[:, 0].max() * np.sin(phi)),
        "magaxis_x": ("phi", equilibrium.magnetic_axis[0] * np.cos(phi)),
        "magaxis_y": ("phi", equilibrium.magnetic_axis[0] * np.sin(phi)),
    }
)

preconstruct the generomak figure background

In [4]:
fig_gen_background = (
    generomak_data.hvplot.line(x="lcfs_x_mid_out", y="lcfs_y_mid_out").opts(color="red")
    * generomak_data.hvplot.line(x="lcfs_x_mid_in", y="lcfs_y_mid_in").opts(color="red")
    * generomak_data.hvplot.line(x="fw_x_mid_out", y="fw_y_mid_out").opts(color="black")
    * generomak_data.hvplot.line(x="fw_x_mid_in", y="fw_y_mid_in").opts(color="black")
    * generomak_data.hvplot.line(x="magaxis_x", y="magaxis_y").opts(color="red", line_dash="dashed")
    * hv.Curve(([ds_cxrs.beam_origin.values[0] * np.cos(ds_cxrs.beam_origin.values[1]),
                 ds_cxrs.beam_point.values[0] * np.cos(ds_cxrs.beam_point.values[1])],
                 [ds_cxrs.beam_origin.values[0] * np.sin(ds_cxrs.beam_origin.values[1]),
                 ds_cxrs.beam_point.values[0] * np.sin(ds_cxrs.beam_point.values[1])],
                 )).opts(color="green", line_dash="dashed")
).opts(xlabel="x [m]", ylabel="y [m]")


# Charge Exchange Recombination Spectroscopy
Is a spectroscopic diagnostic technique based on an analysis of a spectral line shape. For this reason the spectra measurements are taken with high resolution. The spectral line of interest is generated by interaction of a Neutral Beam Heating (NBI) particles (here we use deuterium beam denoted with $D$) with an absolutely stripped ions of light impurity (here denoted with $I$). In a charge exchange collision, the bound electron jumps from a fast deuterium neutral to an excited state in an impurity ion. Consequently, the bound electron  decays into a lower energy state by radiating a photon $h\nu$.

$$ I^{Z} + D^{0} \rightarrow I^{Z-1*} + D^{1+} \rightarrow I^{Z-1} + D^{1+} + h\nu_{p \rightarrow q}$$

This line is referred to as "active line" or "active component", since it has to be actively generated by the NBI. The $p \rightarrow q$ denotes the upper and lower electron energy states which define the energy of the photon $h\nu$ in the radiator's frame. The line you will be working on is the Carbon 5+ 8 -> 7 line with rest wavelength 529.07 nm.

In the laboratory frame, the energy of the radiated photon is a subject to a doppler effect given by the velocity of the radiationg impurity $I^{Z-1*}$. The observed line shape is thus governed by the properties of the velocity distribution function of $I$. The properties of the distribution function an be inferred from the line shape. Here, we assume $I$ has a Maxwell-Boltzmann distribution which gives the line a Gaussian shape. The shift of the line with respect to the rest wavelength can thus be used to calculate the plasma rotation, the width of the line (standard deviation) can be used to obtain temperature. Analysis of multiple channels can be used to reconstruct plasma rotation and ion temperature profiles because the radiation observed by each channel is confined to a small volume given by the beam and channel intersection.

### Nuisances
Due to plasma particle interactions the excited states $I^{Z-1*}$ are generated inside plasma even without the presence of the beam. This is called "passive line" or "passive component". There are several mechanisms how $I^{Z-1*}$ is intrinsically generated in plasma. Each mechanism is is caused by different reactants (electron collisions, background $D{0}$ neutrals) and thus happens in a different plasma shell (given by the density distribution of the reactants). Because the photons are radiated by the same impurity $I^{Z-1*}$ and transition $p \rightarrow q$, in the radiator's frame the photon has the same energy $h\nu_{p \rightarrow q}$ for all mechanisms. However, the observed line shape of the passive component can become complicated. Since the volume it is collected from by the diagnostic depends on the channel and the plasma shells which in turn have evolving temperatures and rotations, the line shape of the passive component can take non-parametric shapes. Because in measured spectrum both active and passive components are superimposed, the analysis of the active component shape can get tricky.

### Beam Chopping for Sanity
The result is, that the passive component of the spectrum is present in every measurement. The active component is present only when the NBI is injected into plasma. One technique to deal with the active-passive component separation is to turn turn on and off the NBI periodically (to "chop" the beam). In reality this provides separate information about the passive component which can be used in inference of the properties of the active component. Although in reality, the plasma conditions evolve and the passive component can change its shape, it your data this phenomenon is negligible.

In [ ]:
ds_cxrs.sel(channel=14, observation=1).hvplot.line(
    x="wavelength", y="ReconstructedRadiance", by="NBI", groupby=[]
).opts(
    xlabel="Wavelength (nm)",
    ylabel="Radiance (W/m^2/sr/nm)",
    title="Beam Chopping",
    width=900,
    height=600,
)

# The Diagnostic
The diagnostic is build from three main parts. The first part are the optics at the Tokamak, defining the way the light generated in the tokamak is collected. The second part are the optical fibres collecting the light focused by the optics and guiding it to a spectroscopic instrumentation usualy tens of meters away. The third part in the spectroscopic instrumentation converting the collected light into measured spectra.
## Optical Setup at the Tokamak
The optics on the tokamak focus light from 32 separate volumes onto a set of optical fibres. The 32 volumes are called "channels". Each channel starts at the position of the optics (same for all channels) and has a different radial position where it intersects the neutral beam. The picture below shows your measurement configuration.

In [ ]:
fig_cxrs_setup = (
    fig_gen_background
    * hv.Scatter((ds_cxrs.spot_x, ds_cxrs.spot_y)).opts(
        color="blue", marker="x", size=10
    )
    * hv.Scatter(
        (
            ds_cxrs.optics.values[0] * np.cos(ds_cxrs.optics.values[1]),
            ds_cxrs.optics.values[0] * np.sin(ds_cxrs.optics.values[1]),
        ),
    ).opts(color="blue", marker="o", size=10)
)
fig_cxrs_setup.opts(xlabel="x [m]", ylabel="y [m]", width=600, height=600)

## Optical Fibres
The optical fibres are susually specifically chosen to match the properties of the optics and the spectroscopic instrumentation. In this example, we ignore many aspects and simulate the presence of optical fibres by the attenuation they affect the collected light with.

## Spectroscopic Instrumentation
In your case the spectroscopic instrumentation is made of an imaging spectrometer and the spectroscopic camera. 

### Spectrometer
The spectrometer takes the light from the optical fibres at its input, breaks it down to a spectra and focuses the light into 32 separate tracks on the spectroscopic camera. During this process, the "virgin" spectrum arriving into the spectrometer, is deformed by the instrumental function of the spectrometer. The process can be described by taking a convolution of the "virgin" spectrum with the instrumental function. To make things a bit simpler, the instrumental function of your spectrometer has a Gaussian shape.

In [ ]:
ds_cxrs.InstrumentalFunction.hvplot.line(x="wavelength", by="spectrometer").opts(
    xlabel="Wavelength [nm]",
    ylabel="Instrumental Function",
    title="Instrumental Function",
    width=900,
    height=600,
)

### Camera
The spectroscopc camera is equipped with a 2D CCD detector. It collects photons from the spectrometer and forms a a set of discrete spectra. During the conversion into the digital output called counts (unit of intensity) a noise is added to the data. Usually, diagnostic officers reconstruct the radiance (here called ReconstructedRadiance) collected by the diagnostic. This is done by applying calibration constants obtained from calibration measurements.  For your convenience, the radiance was reconstructed for you in a very simple way. You don't need to repeat the calibration, if you don't need to improve the noise.

In [ ]:
fig_spectra = ds_cxrs.hvplot.line(x="wavelength", y="CameraCounts", by="NBI", groupby=["channel", "observation"]).opts(
    width=900,
    height=600,
    xlabel="Wavelength [nm]",
    ylabel="Counts",
    title="Camera Counts",
) + ds_cxrs.hvplot.line(x="wavelength", y="ReconstructedRadiance", by="NBI", groupby=["channel", "observation"]).opts(
    width=900,
    height=600,
    xlabel="Wavelength [nm]",
    ylabel="Radiance [W/m^2/sr/nm]",
    title="Reconstructed Radiance",
)
fig_spectra.cols(1)

# System A and B
Because a single spectrometer and camera can accommodate only 16 channels, your CXRS diagnostic is separated into two spectrometers with two different cameras. Both spectrometers and cameras have slightly different instrumental function and noise properties.

# Fitting of the instrumental functions in pymc
The model is built from stochastic priors, deterministic calculations and the observed variable. If we want to save the samples of deterministically calculated variables (in this case of line_shape), we can wrap them in pm.Deterministic. Also, it is a good habit to define the model coordinates and use the defined coordinates in the dims parameter. It will simplify debugging and further work. 

In [9]:
def gauss_with_offset(x, mean, std, height, offset):
    return offset + height * pm.math.exp(-0.5 * ((x - mean) / std) ** 2)


with pm.Model(coords={"spectrometer": ds_cxrs.spectrometer.values, "wavelength": ds_cxrs.wavelength.values}) as model:

    inst_mean = pm.Triangular("inst_mean", lower=529, upper=531, c=530, dims=("spectrometer")) # define prior for the mean of the instrumental function
    inst_std = pm.Uniform("inst_std", lower=0.01, upper=0.02, dims=("spectrometer")) # define prior for the standard deviation of the instrumental function
    inst_height = pm.LogNormal("inst_height", mu=np.log(1e4), sigma=0.2, dims=("spectrometer")) # define prior for the height of the instrumental function
    baseline = pm.Uniform("baseline", lower=290, upper=320, dims=("spectrometer")) # define prior for the baseline of the instrumental function
    noise_std = pm.HalfNormal("noise_std", sigma=80, dims=("spectrometer")) # define prior for the noise of the instrumental function

    wavelength = ds_cxrs.wavelength.values[:, np.newaxis] # define the wavelength variable, it is a 2D array with the same shape as the instrumental function

    line_shape = gauss_with_offset(wavelength, inst_mean, inst_std, inst_height, baseline) # define the line shape as a deterministic variable
    pm.Deterministic("line_shape", line_shape, dims=("wavelength", "spectrometer")) # wrap the line shape in a Deterministic variable. This makes the model keep the samples and generate prior predictive samples

    instrumental_function = pm.Normal("instrumental_function", mu=line_shape, sigma=noise_std, observed=ds_cxrs.InstrumentalFunction.values.T, dims=("wavelength", "spectrometer"))


## Prior Predictive Checks
It is always a good step to do prior predictive checks. This draws model samples from the priors and propagates it through the model. This way we can get prior samples of the "line_shape". The samples can be used to visually check wether our priors can describe the data well enough. It can happen that the selected distributions and ranges are preventing the model to cover the data or that the defined space is too broad which will result in problematic sampling of the posterior.

In [ ]:
# Sample from the prior predictive distribution
with model:
    prior_pred = pm.sample_prior_predictive(random_seed=42)


visually check the model

In [ ]:
def instrumental_ppc_plot(prior_pred):
    prior = prior_pred.prior
    observed = prior_pred.observed_data
    return  (
    hv.Path(
        (
            prior.wavelength.values,
            prior.line_shape.sel(chain=0, spectrometer="A")
            .transpose("wavelength", "draw")
            .values,
        )
    ).opts(color="red", alpha=0.05)
    * hv.Curve(
        (observed.wavelength, observed.instrumental_function.sel(spectrometer="A").values)
    ).opts(color="black")
    + hv.Path(
        (
            prior.wavelength.values,
            prior.line_shape.sel(chain=0, spectrometer="B")
            .transpose("wavelength", "draw")
            .values,
        )
    ).opts(color="green", alpha=0.05)
    * hv.Curve(
        (observed.wavelength, observed.instrumental_function.sel(spectrometer="B").values)
    ).opts(color="black")
)

plot_ppc = instrumental_ppc_plot(prior_pred)
plot_ppc.opts(width=900, height=600)

From the prior predictions we can see, that the mean of the line could be probably selected within a more narrow band. Also, the prior "line_std" governing the line width has to be made boader in order to cover the data. The data also show that the lines have two very different widths, this can be also prescribed to the priors.

In [12]:
with pm.Model(coords={"spectrometer": ds_cxrs.spectrometer.values, "wavelength": ds_cxrs.wavelength.values}) as model2:

    inst_mean = pm.Triangular("inst_mean", lower=529.9, upper=530.1, c=530, dims=("spectrometer")) # define prior for the mean of the instrumental function
    inst_std = pm.Uniform("inst_std", lower=[0.08, 0.03], upper=[0.16, 0.1], dims=("spectrometer")) # define prior for the standard deviation of the instrumental function
    inst_height = pm.Triangular("inst_height", lower=[10000, 20000], upper=[15000, 25000], c=[12500, 22500], dims=("spectrometer")) # define prior for the height of the instrumental function
    baseline = pm.Uniform("baseline", lower=290, upper=350, dims=("spectrometer")) # define prior for the baseline of the instrumental function
    noise_std = pm.HalfNormal("noise_std", sigma=80, dims=("spectrometer")) # define prior for the noise of the instrumental function

    wavelength = ds_cxrs.wavelength.values[:, np.newaxis] # define the wavelength variable, it is a 2D array with the same shape as the instrumental function

    line_shape = gauss_with_offset(wavelength, inst_mean, inst_std, inst_height, baseline) # define the line shape as a deterministic variable
    pm.Deterministic("line_shape", line_shape, dims=("wavelength", "spectrometer")) # wrap the line shape in a Deterministic variable. This makes the model keep the samples and generate prior predictive samples

    instrumental_function = pm.Normal("instrumental_function", mu=line_shape, sigma=noise_std, observed=ds_cxrs.InstrumentalFunction.values.T, dims=("wavelength", "spectrometer"))


In [ ]:
with model2:
    prior_pred2 = pm.sample_prior_predictive(random_seed=42)

In [ ]:
plot_ppc2 = instrumental_ppc_plot(prior_pred2)
plot_ppc2.opts(width=900, height=600)

In [ ]:
with model2:
    trace = pm.sample(mp_ctx="spawn")

In [ ]:
az.plot_trace(trace, var_names=["inst_mean", "inst_std", "inst_height", "baseline", "noise_std"], backend="bokeh")

In [ ]:
with model2:
    posterior_pred = pm.sample_posterior_predictive(trace, var_names=["instrumental_function"], random_seed=42)

In [ ]:
def plot_posterior_pred(posterior_pred):
    return hv.Path(
        (
            (
            posterior_pred.posterior_predictive.wavelength.values,
            posterior_pred.posterior_predictive.instrumental_function.sel(
                chain=0, spectrometer="A"
            )
            .transpose("wavelength", "draw")
            .values,
        )
    )
).opts(color="red", alpha=0.05) * hv.Curve(
    (
        posterior_pred.observed_data.wavelength,
        posterior_pred.observed_data.instrumental_function.sel(spectrometer="A").values,
    )
).opts(color="black", xlabel="Wavelength [nm]", ylabel="Instrumental Function [counts]") + hv.Path(
    (
        (
            posterior_pred.posterior_predictive.wavelength.values,
            posterior_pred.posterior_predictive.instrumental_function.sel(
                chain=0, spectrometer="B"
            )
            .transpose("wavelength", "draw")
            .values,
        )
    )
).opts(color="green", alpha=0.05) * hv.Curve(
    (
        posterior_pred.observed_data.wavelength,
        posterior_pred.observed_data.instrumental_function.sel(spectrometer="B").values,
    )
).opts(color="black", xlabel="Wavelength [nm]", ylabel="Instrumental Function [counts]")

fig_posterior_pred = plot_posterior_pred(posterior_pred)
fig_posterior_pred.opts(width=1000, height=600)

# Student Task: Infer profile of impurity temperature and toroidal rotation from CXRS measurements
## Ion Temperature
For maxwellian plasma the ion temperature can be calculated from the standard deviation of the active component. You can assume the active component to have a Gaussian shape. The temperature is then calculated as:

$$ T_i = \frac{m_i c^2}{e}\left( \frac{\sigma_\lambda}{\lambda_0}\right)$$

## Plasma Rotation
The plasma rotation velocity can be calculated from the doppler shift of the spectral line. Don't forget that you observe the rotation under an angle and the true rotation is projected into your observation. In the ds_cxrs you have all the data to calculate the projection coefficient p for every diagnostic channel.

$$v = c \frac{\Delta \lambda}{\lambda_0}p$$

Don't forget to follow the advices from the lectures, e.g. start simple and work your way up, iterate fast by inferring small subset of channels and use prior predictive.